This notebook parses and analyzes logs generated by victimplay
in the `selfplay/*` folder.

### Load libraries

In [ ]:
import dataclasses

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Load and parse logs

In [ ]:
%%time
with open("../victimplay-training/svm3/selfplay/log20211215-051729-EDF594336F74E22B.log") as f:
    lines = [l.strip() for l in f.readlines()]
len(lines)

In [ ]:
@dataclasses.dataclass
class GameResult:
    raw_line: str
    
    dt_str: str = dataclasses.field(init=False)
    game_idx: int = dataclasses.field(init=False)
    
    victim_color: str = dataclasses.field(init=False)
    adv_color: str = dataclasses.field(init=False)
    
    score_diff: float = dataclasses.field(init=False)
    
    # TODO: parse `adv_nets:...`

    def __post_init__(self):
        self.dt_str = self.raw_line.split(': Game #')[0]
        self.game_idx = int(self.raw_line.split(': Game #')[1].split(' ')[0])
        
        self.victim_color = self.raw_line.split('victim (')[1][0]
        self.adv_color = self.raw_line.split('adv (')[1][0]
        assert {self.victim_color, self.adv_color} == {"B", "W"}
        
        self.score_diff = float(self.raw_line.split('score: ')[1].split(';')[0])
        
    def as_nice_dict(self):
        ret_dict = dataclasses.asdict(self)
        ret_dict.pop("raw_line")
        return ret_dict

In [ ]:
%%time
GAME_RESULTS = [
    GameResult(l)
    for l in lines
    if "score: " in l
]
len(GAME_RESULTS)

In [ ]:
%%time
df = pd.DataFrame(
    [gr.as_nice_dict() for gr in GAME_RESULTS]
)
df.dt_str = pd.to_datetime(df.dt_str)
df.head()

### Analyze results

In [ ]:
plt.figure(dpi=150)

roll_sz = 2000
#df.score_diff.rolling(roll_sz).mean().plot(label="score_diff")
(-df[df.victim_color == "W"].score_diff).rolling(roll_sz).mean().plot(label="score_diff (adv=B)", alpha=0.8)
(-df[df.victim_color == "B"].score_diff).rolling(roll_sz).mean().plot(label="score_diff (adv=W)", alpha=0.8)

plt.legend();
plt.xlabel("Game #")
plt.ylabel(f"Adv - Victim score diff");
plt.title(f"Adv - Victim score diff (rolling avg. w={roll_sz})");

In [ ]:
plt.figure(dpi=150)

roll_sz = 2000
#df.score_diff.rolling(roll_sz).mean().plot(label="score_diff")
(df[df.victim_color == "W"].score_diff < 0).rolling(roll_sz).mean().plot(label="win_rate (adv=B)", alpha=0.8)
(df[df.victim_color == "B"].score_diff < 0).rolling(roll_sz).mean().plot(label="win_rate (adv=W)", alpha=0.8)

plt.legend();
plt.xlabel("Game #")
plt.ylabel(f"Adv. win rate");
plt.title(f"Adv. win rate (rolling avg. w={roll_sz})");

In [ ]:
plt.figure(dpi=150, figsize=(8, 2))

plt.subplot(1, 2, 1)
(-df[df.victim_color == "W"].score_diff).hist(bins=100, alpha=1);
plt.title("Adv(B) - Victim score distribution");

plt.subplot(1, 2, 2)
(-df[df.victim_color == "B"].score_diff).hist(bins=100, color="tab:orange");
plt.title("Adv(W) - Victim score distribution");